# ERC-4626: vault ecosystem comparison across chains

This notebook serves both as a coding tutorial and a useful data analytics tool for ERC-4626 vaults. 

- In this notebook, we examine different ERC-4626 vaults across different EVM blockchains   
    - Currently we do not scan non-ERC-4626 vaults like Enzyme Finance, or any protocol-native vaults like Hyperliquid HPL. This is not an inherit limitation, this is not just yet implemented.
- We assemble various data tables out of the vault data to show and compare the blockchain ecosystems
- The analysis focus on USD-stablecoin nonminatd vaults
    - Currently missing are e.g. WETH vaults and staking vaults for various small cap tokens
    - There is no ERC standard for vaults fees - for some protocols we have manualled added fee reading support  
- The list of chains is somewhat randomly selected and very easy to extend to contain any chain supported by [Envio's HyperSync](https://docs.envio.dev/docs/HyperSync/hypersync-supported-networks)
- Everything is open source: You can run this notebook and associated scripts yourself on your local computer, it will take around an hour

## Usage

- Read general instructions how to run the package tutorials
- See `ERC-4626 scanning all vaults onchain` example in tutorials first how to build a vault database as local `vault_db.pickle` file.




## Setup

- Set up notebook renderinb parmaeters

In [89]:
import pandas as pd

pd.options.display.float_format = '{:,.2f}'.format



## Read scanned data

- Read the Pickle database our scanning script produced earlier 

In [90]:
import pickle
from pathlib import Path

import pandas as pd

from eth_defi.token import is_stablecoin_like

output_folder = Path("~/.tradingstrategy/vaults").expanduser()
vault_db = output_folder / "vault-db.pickle"
assert vault_db.exists(), "Run the vault scanner script first"

vault_db = pickle.load(open(vault_db, "rb"))

print(f"We have data for {len(vault_db)} vaults")

We have data for 6983 vaults


## Transform data

- Prepare the raw vault pickled data as Pandas DataFrame for data research

In [91]:
import datetime
from pprint import pformat
import pandas as pd
from eth_defi.erc_4626.hypersync_discovery import ERC4262VaultDetection
from eth_defi.chain import get_chain_name
from eth_defi.token import is_stablecoin_like

data = list(vault_db.values())
df = pd.DataFrame(data)

# print("Raw row example:")
# print(df.iloc[0])

# Build useful columns out of raw pickled Python data
# _detection_data contains entries as ERC4262VaultDetection class
entry: ERC4262VaultDetection
df["Chain"] = df["_detection_data"].apply(lambda entry: get_chain_name(entry.chain))
df["Protocol identified"] = df["_detection_data"].apply(lambda entry: entry.is_protocol_identifiable())
df["Stablecoin denominated"] = df["_denomination_token"].apply(lambda token_data: is_stablecoin_like(token_data["symbol"]) if pd.notna(token_data) else False)
df["ERC-7540"] = df["_detection_data"].apply(lambda entry: entry.is_erc_7540())
df["Fee detected"] = df.apply(lambda row: (row["Mgmt fee"] is not None) or (row["Perf fee"] is not None), axis=1)
# Event counts
df["Deposit count"] = df["_detection_data"].apply(lambda entry: entry.deposit_count)
df["Redeem count"] = df["_detection_data"].apply(lambda entry: entry.redeem_count)
df["Total events"] = df["Deposit count"] + df["Redeem count"]
df["Age"] = datetime.datetime.utcnow() - df["First seen"]
df["NAV"] = df["NAV"].astype("float64")
df = df.sort_values(["Chain", "Address"])
df = df.set_index(["Chain", "Address"])

print("DataFrame MultiIndex is:", ", ".join(x for x in df.index.names))
print("DataFrame columns are:", ", ".join(x for x in df.columns))

display(df.head())

DataFrame MultiIndex is: Chain, Address
DataFrame columns are: Symbol, Name, Denomination, NAV, Protocol, Mgmt fee, Perf fee, Shares, First seen, _detection_data, _denomination_token, _share_token, Protocol identified, Stablecoin denominated, ERC-7540, Fee detected, Deposit count, Redeem count, Total events, Age


Symbol  \
Chain    Address                                                               
Arbitrum 0x000000f0C01c6200354f240000b7003668B4D080            vMAIA-bHERMES   
         0x00003b020004328e005A0011b99a00c100CB9040            vMAIA-bHERMES   
         0x0021f89457A5DD4F709c68A2Baa2CA94a4D2acfF  wrappedConvexCrvusdUsdt   
         0x0022228a2cc5E7eF0274A7Baa600d44da5aB5776                    stUSD   
         0x004626A008B1aCdC4c74ab51644093b155e59A23                    stEUR   

                                                                                                  Name  \
Chain    Address                                                                                         
Arbitrum 0x000000f0C01c6200354f240000b7003668B4D080  Vote Maia - Burned Hermes: Aggregated Gov + Yi...   
         0x00003b020004328e005A0011b99a00c100CB9040  Vote Maia - Burned Hermes: Aggregated Gov + Yi...   
         0x0021f89457A5DD4F709c68A2Baa2CA94a4D2acfF                         Wrapped Convex crvUSD/USDT   
         0x0022228a2cc5E7eF0274A7Baa600d44da5aB5776                                        Staked USDA   
         0x004626A008B1aCdC4c74ab51644093b155e59A23                                        Staked EURA   

                                                    Denomination        NAV  \
Chain    Address                                                              
Arbitrum 0x000000f0C01c6200354f240000b7003668B4D080         MAIA  83,522.99   
         0x00003b020004328e005A0011b99a00c100CB9040         MAIA  89,756.42   
         0x0021f89457A5DD4F709c68A2Baa2CA94a4D2acfF   cvxcrvUSDT       0.00   
         0x0022228a2cc5E7eF0274A7Baa600d44da5aB5776         USDA 913,064.41   
         0x004626A008B1aCdC4c74ab51644093b155e59A23         EURA 304,672.49   

                                                           Protocol Mgmt fee  \
Chain    Address                                                               
Arbitrum 0x000000f0C01c6200354f240000b7003668B4D080  <generic 4626>     None   
         0x00003b020004328e005A0011b99a00c100CB9040  <generic 4626>     None   
         0x0021f89457A5DD4F709c68A2Baa2CA94a4D2acfF  <generic 4626>     None   
         0x0022228a2cc5E7eF0274A7Baa600d44da5aB5776  <generic 4626>     None   
         0x004626A008B1aCdC4c74ab51644093b155e59A23  <generic 4626>     None   

                                                    Perf fee  \
Chain    Address                                               
Arbitrum 0x000000f0C01c6200354f240000b7003668B4D080     None   
         0x00003b020004328e005A0011b99a00c100CB9040     None   
         0x0021f89457A5DD4F709c68A2Baa2CA94a4D2acfF     None   
         0x0022228a2cc5E7eF0274A7Baa600d44da5aB5776     None   
         0x004626A008B1aCdC4c74ab51644093b155e59A23     None   

                                                                        Shares  \
Chain    Address                                                                 
Arbitrum 0x000000f0C01c6200354f240000b7003668B4D080   83522.991343137221650124   
         0x00003b020004328e005A0011b99a00c100CB9040   89756.422928390178159298   
         0x0021f89457A5DD4F709c68A2Baa2CA94a4D2acfF                5.00187E-13   
         0x0022228a2cc5E7eF0274A7Baa600d44da5aB5776   820069.86866595125579169   
         0x004626A008B1aCdC4c74ab51644093b155e59A23  283620.131046566361981316   

                                                             First seen  \
Chain    Address                                                          
Arbitrum 0x000000f0C01c6200354f240000b7003668B4D080 2024-10-17 02:54:55   
         0x00003b020004328e005A0011b99a00c100CB9040 2024-08-15 12:10:17   
         0x0021f89457A5DD4F709c68A2Baa2CA94a4D2acfF 2024-09-17 13:21:44   
         0x0022228a2cc5E7eF0274A7Baa600d44da5aB5776 2024-01-11 08:32:25   
         0x004626A008B1aCdC4c74ab51644093b155e59A23 2023-09-01 10:47:47   

                                                                                       _detection_dat

## Vaults per chain summary

- Get a summary of scanned chains at what vaults they have
- *Generic* status means that we do not have classification rules to determine the protocol on which a particular ERC-4626 vault belongs
- *Broken* status means that we could not correctly extract ERC-4626 information out of a smart contract

To detect the protocol of a vault, we need to maintain a [manual rule list here](https://github.com/tradingstrategy-ai/web3-ethereum-defi/blob/master/eth_defi/erc_4626/classification.py). Not all protocols are supported at the moment. as there are too many protocols to manually examine and identify them. Open source contributions welcome.




In [92]:
nav_threshold = 10_000

# Built different masks
identified_filter = df["Protocol identified"] == True
stablecoin_denominated = df["Stablecoin denominated"] == True
notable_nav = df["Stablecoin denominated"] & (df["NAV"] >= nav_threshold)
notable_usage = df["Stablecoin denominated"] & (df["NAV"] >= nav_threshold)
erc_7540 = df["ERC-7540"] == True 
fee_detected = df["Fee detected"] == True 

# Create the summary DataFrame
summary_df = pd.DataFrame({
    'Total vaults detected': df.groupby(level='Chain').size(),
    'Protocol correctly identified': df[identified_filter].groupby(level='Chain').size(),
    'Stablecoin denominated': df[stablecoin_denominated].groupby(level='Chain').size(),
    f'Notable stablecoin NAV (min {nav_threshold} USD)': df[notable_nav].groupby(level='Chain').size(),
    f'ERC-7540': df[erc_7540].groupby(level='Chain').size(),
    f'Fee data supported': df[fee_detected].groupby(level='Chain').size(),
}).fillna(0).astype(int)

print("Vault counts per feature per chain")
display(summary_df)

Vault counts per feature per chain


,Total vaults detected,Protocol correctly identified,Stablecoin denominated,Notable stablecoin NAV (min 10000 USD),ERC-7540,Fee data supported
Chain,,,,,,
Arbitrum,1913,240,654,118,0,231
Avalanche,251,8,56,2,0,48
Base,1145,478,416,74,0,93
Berachain,225,1,17,14,0,2
Binance,315,9,84,11,0,25
Ethereum,2108,237,697,240,0,195
Hyperliquid,11,0,5,0,0,1
Mantle,37,0,11,0,0,5
Mode,56,0,26,2,0,0


# Vault deployment history

- Show how much history we have for each chain


In [93]:
# Assuming your DataFrame is named 'df'
seen_df = df.groupby(level='Chain')['First seen'].agg(['min', 'max']).reset_index()

# Rename columns for clarity
seen_df.columns = ['Chain', 'First vault deployed', 'Last vault deployed']

display(seen_df)

,Chain,First vault deployed,Last vault deployed
0,Arbitrum,2022-03-28 09:03:11,2025-04-03 20:38:18
1,Avalanche,2022-04-11 00:05:24,2025-04-03 10:13:13
2,Base,2023-08-04 15:36:07,2025-04-04 20:00:19
3,Berachain,2025-01-26 00:06:03,2025-04-05 15:02:12
4,Binance,2022-05-27 17:25:18,2025-03-31 18:19:30
5,Ethereum,2019-06-11 06:17:19,2025-04-07 10:13:47
6,Hyperliquid,2025-02-19 16:29:00,2025-04-05 17:21:00
7,Mantle,2023-08-22 13:25:16,2024-11-29 08:55:34
8,Mode,2024-03-06 23:58:43,2025-01-03 19:23:51
9,Polygon,2022-03-30 14:21:39,2025-04-07 09:21:54


## Largest USD vaults

- Show the stablecoin-denominated vaults across different chains that have largest USD treasury 

In [94]:
largest_threshold = 20
largest_df = df.reset_index()
# Filter out crap
largest_df = largest_df[largest_df["Total events"] > 100] 
largest_df = largest_df[largest_df["Stablecoin denominated"] == True] 
largest_df = largest_df.sort_values(["NAV"], ascending=False)

largest_df = largest_df[["Chain", "Address", "Name", "Denomination", "NAV", "Total events"]]

display(largest_df.head(largest_threshold))


,Chain,Address,Name,Denomination,NAV,Total events
5529,Ethereum,0xa3931d71877C0E7a3148CB7Eb4463524FEc27fbD,Savings USDS,USDS,"3,024,741,408.37",27360
5124,Ethereum,0x9D39A5DE30e57443BfF2A8307A4256c8797A3497,Staked USDe,USDe,"2,222,459,913.31",69024
5046,Ethereum,0x90D2af7d622ca3141efA4d8f1F24d86E5974Cc8F,Ethereal Pre-deposit Vault,USDe,"913,360,679.60",47505
4943,Ethereum,0x83F20F44975D03b1b09e64809B757c47f942BEeA,Savings Dai,DAI,"521,432,390.75",72232
3432,Berachain,0x90bc07408f5b5eAc4dE38Af76EA6069e1fcEe363,Bridged USDC (Stargate)Vault,USDC.e,"409,532,404.18",122689
5132,Ethereum,0x9Fb7b4477576Fe5B32be4C1843aFB1e55F251B33,Fluid USD Coin,USDC,"232,951,751.01",17396
5765,Ethereum,0xd63070114470f685b75B74D60EEc7c1113d33a3D,Usual Boosted USDC,USDC,"202,226,401.53",17290
3696,Binance,0x7788A3538C5fc7F9c7C8A74EAC4c898fC8d87d92,Staked USDX,USDX,"199,682,796.15",5404
4610,Ethereum,0x5C20B550819128074FD538Edf79791733ccEdd18,Fluid Tether USD,USDT,"183,504,834.50",10297
5269,Ethereum,0xBEEF01735c132Ada46AA9aA4c54623cAA92A64CB,Steakhouse USDC,USDC,"130,886,157.02",4068


## Largest USD vault per chain

- Get the largest vault of each chain

In [95]:
# Get the index of max NAV for each chain
max_nav_idx = largest_df.groupby('Chain')['NAV'].idxmax()
# Use these indices to get the full rows
result = largest_df.loc[max_nav_idx]
display(result)

,Chain,Address,Name,Denomination,NAV,Total events
192,Arbitrum,0x1A996cb54bb95462040408C06122D45D6Cdb6096,Fluid USD Coin,USDC,"28,393,019.48",33392
2048,Avalanche,0x8475509d391e6ee5A8b7133221CE17019D307B3E,HiYield Treasury Bill Vault,USDC,"2,274,708.15",177
2713,Base,0x7BfA7C4f149E7415b73bdeDfe609237e29CBF34A,Spark USDC Vault,USDC,"75,161,699.89",5566
3432,Berachain,0x90bc07408f5b5eAc4dE38Af76EA6069e1fcEe363,Bridged USDC (Stargate)Vault,USDC.e,"409,532,404.18",122689
3696,Binance,0x7788A3538C5fc7F9c7C8A74EAC4c898fC8d87d92,Staked USDX,USDX,"199,682,796.15",5404
5529,Ethereum,0xa3931d71877C0E7a3148CB7Eb4463524FEc27fbD,Savings USDS,USDS,"3,024,741,408.37",27360
6027,Mode,0x882fD369341FC435ad5E54e91d1ebC23b1Fc6d4C,USDC Ironclad Vault,USDC,"337,647.27",183
6515,Polygon,0x781FB7F6d845E3bE129289833b04d43Aa8558c42,Compound USDC,USDC,"22,043,462.31",4022
6980,Unichain,0xE5565daeE2ccDD18736AD8B1A279A43626bbf369,POPT-V1 USDC LP on USDC/WETH 5bps,USDC,"23,465.24",287


## Most active vaults across all chains

- Determine vault activity by number of deposit and redeem events
- Based on all-time event count, not recent event count 
- Events may be driven by bots, so this may not reflect the popularity of a vault amount users


In [96]:
largest_threshold = 20
largest_df = df.reset_index().sort_values(["Total events"], ascending=False)

largest_df = largest_df[["Chain", "Address", "Name", "Denomination", "NAV", "Age", "Total events", "Deposit count", "Redeem count"]]

display(largest_df.head(largest_threshold))

,Chain,Address,Name,Denomination,NAV,Age,Total events,Deposit count,Redeem count
6648,Polygon,0xA013Fbd4b711f9ded6fB09C1c0d358E2FbC2EAA0,USDC yVault-A,USDC,"573,240.60",528 days 20:08:28.060484,490156,465206,24950
6697,Polygon,0xBb287E6017d3DEb0e2E65061e8684eab21060123,USDT yVault-A,USDT,"462,505.51",486 days 19:51:56.060484,458197,407689,50508
5388,Ethereum,0xD9A442856C234a39a81a089C06451EBAa4306a72,pufETH,WETH,"69,741.74",431 days 17:09:41.060484,167786,151517,16269
3098,Base,0xa0E430870c4604CcfC7B38Ca7845B1FF653D0ff1,Moonwell Flagship ETH,WETH,"14,427.04",298 days 23:57:01.060484,140123,85196,54927
2211,Base,0x0B0193fAD49DE45F5E2B0A9f5D6Bc3BB7D281688,FARM_WETH,WETH,"1,188.19",578 days 12:27:13.060484,134001,76051,57950
3432,Berachain,0x90bc07408f5b5eAc4dE38Af76EA6069e1fcEe363,Bridged USDC (Stargate)Vault,USDC.e,"409,532,404.18",71 days 12:48:13.060484,122689,80101,42588
3,Arbitrum,0x0022228a2cc5E7eF0274A7Baa600d44da5aB5776,Staked USDA,USDA,"913,064.41",452 days 04:21:51.060484,121737,119729,2008
3165,Base,0xc1256Ae5FF1cf2719D4937adb3bbCCab2E00A2Ca,Moonwell Flagship USDC,USDC,"26,677,249.72",298 days 23:57:01.060484,91762,53054,38708
3494,Berachain,0xE59AB0C3788217e48399Dae3CD11929789e4d3b2,Beraborrow iBGT,iBGT,"46,643.06",31 days 00:48:57.060484,79308,43098,36210
2598,Base,0x616a4E1db48e22028f6bbf20444Cd3b8e3273738,Seamless USDC Vault,USDC,"34,525,109.85",83 days 13:20:05.060484,73674,42078,31596


## Most active vault per chain

- Display the number one vault per chain

In [97]:
most_active_df = df.reset_index()

most_active_df = most_active_df[["Chain", "Address", "Name", "Denomination", "NAV", "Age", "Total events", "Deposit count", "Redeem count"]]

# Force thousand separator
most_active_df["Total events"] = most_active_df["Total events"].astype("float64")

max_nav_idx = most_active_df.groupby('Chain')['Total events'].idxmax()
# Use these indices to get the full rows
result = most_active_df.loc[max_nav_idx]

display(result)

,Chain,Address,Name,Denomination,NAV,Age,Total events,Deposit count,Redeem count
3,Arbitrum,0x0022228a2cc5E7eF0274A7Baa600d44da5aB5776,Staked USDA,USDA,"913,064.41",452 days 04:21:51.060484,"121,737.00",119729,2008
2073,Avalanche,0xA25EaF2906FA1a3a13EdAc9B9657108Af7B703e3,GoGoPool Liquid Staking Token,WAVAX,"665,536.08",732 days 15:43:31.060484,"66,102.00",41116,24986
3098,Base,0xa0E430870c4604CcfC7B38Ca7845B1FF653D0ff1,Moonwell Flagship ETH,WETH,"14,427.04",298 days 23:57:01.060484,"140,123.00",85196,54927
3432,Berachain,0x90bc07408f5b5eAc4dE38Af76EA6069e1fcEe363,Bridged USDC (Stargate)Vault,USDC.e,"409,532,404.18",71 days 12:48:13.060484,"122,689.00",80101,42588
3570,Binance,0x1c3f35F7883fc4Ea8C4BCA1507144DC6087ad0fb,kUSDT,VUSD,"2,973,097.94",665 days 05:45:33.060484,"42,081.00",25232,16849
5388,Ethereum,0xD9A442856C234a39a81a089C06451EBAa4306a72,pufETH,WETH,"69,741.74",431 days 17:09:41.060484,"167,786.00",151517,16269
5958,Hyperliquid,0x2831775cb5e64B1D892853893858A261E898FbEb,wHYPE,WHYPE,"165,828.90",25 days 14:43:16.060484,465.00,355,110
5989,Mantle,0x8529019503c5BD707d8Eb98C5C87bF5237F89135,Karak - mETH,mETH,490.37,348 days 18:42:10.060484,"13,021.00",8038,4983
6056,Mode,0xd60DD6981Ec336fDa40820f8cA5E99CD17dD25A0,Renzo aggregator,WETH,232.02,396 days 12:55:33.060484,"21,413.00",12185,9228
6648,Polygon,0xA013Fbd4b711f9ded6fB09C1c0d358E2FbC2EAA0,USDC yVault-A,USDC,"573,240.60",528 days 20:08:28.060484,"490,156.00",465206,24950


## Oldest vaults

- Show oldest vaults

In [100]:
threshold = 1_000

oldest_df = df.reset_index()

oldest_df = oldest_df[["Chain", "Address", "Name", "Age", "Denomination", "NAV", "Total events"]]

# Force thousand separator
oldest_df["Total events"] = oldest_df["Total events"].astype("float64")

# Force event threshold to filter out some crap
oldest_df = oldest_df[oldest_df["Total events"] >= threshold]

max_nav_idx = oldest_df.groupby('Chain')['Age'].idxmax()
# Use these indices to get the full rows
result = oldest_df.loc[max_nav_idx]

display(result)

,Chain,Address,Name,Age,Denomination,NAV,Total events
1523,Arbitrum,0xF46Ce0C13577232D5F29D9Bd78a9Cab278755346,Jones ETH,1045 days 21:39:10.060484,WETH,18.00,"2,335.00"
2070,Avalanche,0x9dd17F32Fc8355aE37425F475A10Cc7BEC8CA36A,,1089 days 19:56:54.060484,,0.00,"1,425.00"
3182,Base,0xc7548d8D7560f6679e369d0556C44Fe1EDdea3E9,FARM_WETH,584 days 21:00:23.060484,WETH,0.97,"1,117.00"
3432,Berachain,0x90bc07408f5b5eAc4dE38Af76EA6069e1fcEe363,Bridged USDC (Stargate)Vault,71 days 12:48:13.060484,USDC.e,"409,532,404.18","122,689.00"
3550,Binance,0x0F8754b36a767C5579178bd8a04D2fCd9D530b70,ygNRCH,1014 days 16:01:37.060484,NRCH,"1,188,908.73","1,195.00"
4922,Ethereum,0x815C23eCA83261b6Ec689b60Cc4a58b54BC24D8D,vTHOR,1078 days 01:28:39.060484,THOR,"79,952,941.55","19,563.00"
5989,Mantle,0x8529019503c5BD707d8Eb98C5C87bF5237F89135,Karak - mETH,348 days 18:42:10.060484,mETH,490.37,"13,021.00"
6056,Mode,0xd60DD6981Ec336fDa40820f8cA5E99CD17dD25A0,Renzo aggregator,396 days 12:55:33.060484,WETH,232.02,"21,413.00"
6495,Polygon,0x73958d46B7aA2bc94926d8a215Fa560A5CdCA3eA,Wrapped Aave Polygon GHST,1069 days 09:07:27.060484,aPolGHST,"1,144,966.22","14,509.00"
